# Developing multiple-shooting in Sun-B1 dynamics

2022.11.23

In [ ]:
using DifferentialEquations
using Plots
using LinearAlgebra
import ForwardDiff
import DiffResults
using AstrodynamicsBase
# import joptimise
using Printf
using JSON

In [ ]:
include("../../julia-r3bp/R3BP/src/R3BP.jl")
# using R3BP
include("../src/SailorMoon.jl")   # relative path to main file of module
param3b = SailorMoon.dyanmics_parameters()


In [ ]:
plotly()

In [ ]:
function plot_circle(radius, x, y, n=100)
    circle = zeros(2,n)
    thetas = LinRange(0.0, 2π, n)
    for i = 1:n
        circle[1,i] = radius*cos(thetas[i]) + x
        circle[2,i] = radius*sin(thetas[i]) + y
    end
    return circle
end

In [ ]:
moon = plot_circle(1-param3b.mu2, param3b.as , 0.0)
earth = plot_circle(param3b.mu2, param3b.as, 0.0)
earth_ub = plot_circle(param3b.mu2+6375/param3b.lstar, param3b.as, 0.0)
earth_lb = plot_circle(param3b.mu2-6375/param3b.lstar, param3b.as, 0.0)

moon_soi_outer = plot_circle(1-param3b.mu2+66000/param3b.lstar, param3b.as, 0.0)

In [ ]:
# solver settings within fitness function
# https://diffeq.sciml.ai/stable/solvers/dynamical_solve/#Symplectic-Integrators
#method = RK4()  # CalvoSanz4()
#reltol = 1e-10
#abstol = 1e-10
dt = 0.001

param3b = SailorMoon.dyanmics_parameters()
lps = SailorMoon.lagrange_points(param3b.mu2)

lp = 2
Az_km = 1200.0
println("Halo guess Az_km: $Az_km")
northsouth = 3   # 1 or 3
guess0 = R3BP.halo_analytical_construct(param3b.mu2, lp, Az_km, param3b.lstar, northsouth)
res = R3BP.ssdc_periodic_xzplane([param3b.mu2,], guess0.x0, guess0.period, fix="period")

x0_stm = vcat(res.x0, reshape(I(6), (6^2,)))[:]
prob_cr3bp_stm = ODEProblem(R3BP.rhs_cr3bp_svstm!, x0_stm, res.period, (param3b.mu2))
sol = solve(prob_cr3bp_stm, Tsit5(), reltol=1e-12, abstol=1e-12)#, saveat=LinRange(0, period, n+1))
monodromy = R3BP.get_stm(sol, 6)   # get monodromy matrix
ys0 = R3BP.get_eigenvector(monodromy, true, 1);

# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

In [ ]:
tmax_si = 280e-3 * 4   # N
isp_si = 1800   # sec
mdot_si = tmax_si / (isp_si * 9.81)
mstar = 2500  # kg
rp_parking = (6378+200)/param3b.lstar   # parking orbit radius

tmax = AstrodynamicsBase.dimensional2canonical_thrust(
    tmax_si, mstar, param3b.lstar, param3b.tstar
)
mdot = AstrodynamicsBase.dimensional2canonical_mdot(
    mdot_si, mstar, param3b.tstar
)

In [ ]:
t_sidereal = 27.3217   * 86400      # sec
t_synodic  = 29.530588 * 86400      # sec
ωb = 2*pi*(t_synodic - t_sidereal)/t_synodic/t_sidereal*param3b.tstar
ωM = 2*pi / (t_synodic / param3b.tstar)
θsf = 2.862339973 
θmf = pi - θsf  # placeholder
tof = 25.981
# println(θm0)
ϕ0 = 0.0
params = [
    param3b.mu2, param3b.mus, param3b.as, θmf, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]    

θmf = π - θsf                
# arrival LPO object
LPOArrival = SailorMoon.CR3BPLPO2(
    res.x0, res.period, ys0, prob_cr3bp_stm, 1e-6, 1e-6, Tsit5(), 1e-12, 1e-12, 0.005
);

xf = SailorMoon.set_terminal_state2(ϕ0, θmf, param3b, LPOArrival)
# in Sun-B1 frame
u0_test = vcat(SailorMoon.transform_EMrot_to_SunB1(xf, θsf, param3b.oms, param3b.as), 1.0)
println(u0_test)

_prob_base = ODEProblem(SailorMoon.rhs_bcr4bp_sb1frame2!, u0_test, [0, -tof], params);
print(params)

In [ ]:
sol_ = SailorMoon.integrate_rk4(_prob_base, 0.001);
print(sol_.u[end])

In [ ]:
pcart = plot(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], label="backward(from LLO):0.001", set_aspect=:equal, size=(700,700))
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

u_midpoint = sol_.u[14000]
scatter!(pcart, [u_midpoint[1]], [u_midpoint[2]], marker=:circle)

pcart

### Check the time-reversibility of integral 

In [ ]:
θm0 = θmf + ωM * (-tof)
params = [
    param3b.mu2, param3b.mus, param3b.as, θm0, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]   
println(params)

u0_test = [
    388.83725006664025, 
    -0.008060103011816226, 
    1.7345154878917807e-24, 
    -1.4287345247061145, 
    10.131022000599954, 
    6.151029489849641e-21, 
    1.2476475049907993
]

_prob = remake(_prob_base, u0=u0_test, t=[0,tof], p=params)

sol_2 = SailorMoon.integrate_rk4(_prob, 0.0001);

In [ ]:
plot!(pcart, hcat(sol_2.u...)[1,:], hcat(sol_2.u...)[2,:], color=:green, label="forward(from LEO): 0.0001")

In [ ]:
sol_tsit5 = solve(_prob, Tsit5(), reltol=1e-12, abstol=1e-12)
plot!(pcart, Array(sol_tsit5)[1,:], Array(sol_tsit5)[2,:], color=:red, linewidth=1.0, label="sol", linestyle=:solid)


In [ ]:

θm0 = θmf + ωM * (-tof)
params = [
    param3b.mu2, param3b.mus, param3b.as, θmf, param3b.oml, param3b.omb, 0.0, 0.0, 0.0, 0.0, 0.0,
    SailorMoon.dv_sun_dir_angles
]   

u0_test = [
    388.83725006664025, 
    -0.008060103011816226, 
    1.7345154878917807e-24, 
    1.4287345247061145, 
    -10.131022000599954, 
    -6.151029489849641e-21, 
    1.2476475049907993
]

_prob = remake(_prob_base, u0=u0_test, t=[0,tof], p=params)

sol_3 = SailorMoon.integrate_rk4(_prob, 0.001);

In [ ]:
plot!(pcart, hcat(sol_3.u...)[1,:], hcat(sol_3.u...)[2,:], color=:red, label="forward(from LEO):0.001")

### Develop propagation function

In [ ]:
n_arc = 5

In [ ]:
# ballistic time right after launch
ballistic_time = 1*86400 / param3b.tstar
ballistic_time_back = 1*86400 / param3b.tstar

dt = 0.001
rp_parking = (6378+200)/param3b.lstar

In [ ]:
function unpack_x(x::AbstractVector{T}, verbose::Bool=false) where T
    # unpack
    nx = length(x)
    x_LEO = x[1:4+3n_arc]
    x_mid = x[5+3n_arc:13+9n_arc]    # x[5+3n_arc:4+3n_arc+9+6n_arc]
    x_LPO = x[14+9n_arc:17+12n_arc]  # x[14+9n_arc:13+9n_arc+4+3n_arc]
    
    # get time of flights
    tofs = [x_LEO[4], x_mid[8], x_mid[9], x_LPO[4]]
    θf = x_LPO[1]
    θs = [
        θf - param3b.oms*sum(broadcast(abs, tofs)),
        θf - param3b.oms*sum(broadcast(abs, tofs[3:4])),
        θf
    ]
    # print message
    if verbose
        @printf("ToF per arc  : %3.3f, %3.3f, %3.3f, %3.3f\n", tofs...)
        @printf("Phase angles : %3.3f, %3.3f, %3.3f\n", θs...)
        println("θf: ", θf)
    end
    return x_LEO, x_mid, x_LPO, tofs, θs
end

In [ ]:
function get_LEO_state(x_LEO, θs, verbose::Bool=false)
    # construct initial state
    sma = (rp_parking + x_LEO[1])/2
    println(sma, " ", rp_parking, " ", x_LEO[1])
    ecc = (x_LEO[1] - rp_parking)/(x_LEO[1] + rp_parking)
    sv0_kep = [sma, ecc, 0.0, x_LEO[2], 0.0, 0.0]  # Ω should be around π
    sv0_i = AstrodynamicsBase.kep2cart(sv0_kep, param3b.mu1)
    
    # check values in SI frame...
    muE = 3.986004418e5
    DV = sqrt(-muE/(sma*param3b.lstar)+2*muE/(rp_parking*param3b.lstar)) - sqrt(muE/(rp_parking*param3b.lstar))
    v_sc_leo = sqrt(muE/rp_parking/param3b.lstar)
    println("DV = $DV km/s  ", "LEO SC velocity: $v_sc_leo km/s")
    sv0_i_original = [sv0_i[1:3].*param3b.lstar, sv0_i[4:6].*param3b.lstar./param3b.tstar]
    println("svo_i_original: $sv0_i_original")
#    sv0_i = vcat(sv0_i_original[1]./ param3b.lstar, sv0_i_original[2] ./ param3b.lstar .*param3b.tstar)
    
    
    sv0_em = SailorMoon.transform_earthIne_to_EMrot(
        sv0_i, x_LEO[2], param3b.oml, param3b.mu2
    )
    print("sv0_i", sv0_i)
    
    #inertial2rotating(sv0_i, θs[1], 1.0) + [-param3b.mu2,0,0,0,0,0]
    # transform to Sun-B1 frame
    θs[1] = 3.14
    sv0_sunb1 = vcat(
        SailorMoon.transform_EMrot_to_SunB1(sv0_em, θs[1], param3b.oms, param3b.as),
        x_LEO[3]   # m0
    )
    
    println("theta_s_initial: ", θs[1])
    
    
    if verbose
        println("SMA [km]: ", sma*param3b.lstar)
        println("ra  [km]: ", x_LEO[1]*param3b.lstar)
        println("Energy in inertial frame: ", -(param3b.mu2)/(2sma))
        println("sv0_sb1: ", (sv0_sunb1[1:3]-[param3b.as,0,0])*param3b.lstar, sv0_sunb1[4:6]*param3b.lstar/param3b.tstar)
    end
    
    # correct answer
#     sv0_sunb1 = [388.8372500765052, -0.008060173712891345, 1.450699321283059e-24, 1.428715370999674, -10.131026038282567, -5.1446746392175664e-21, x_LEO[3]]
    
    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, pi-θs[1], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_sun_dir_angles
    ]
    
    _prob = remake(_prob_base; tspan=[0,ballistic_time], u0 = sv0_sunb1, p = params)
    sol_ballistic_fwd = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[1] + param3b.oms*sol_ballistic_fwd.t[end]

    
    return sol_ballistic_fwd.u[end], θ_iter, sol_ballistic_fwd
end

In [ ]:
param3b.mus

In [ ]:
function get_LPO_state(x_LPO, θs, verbose::Bool=false)
    # propagate the periodic orbit until ϕ*Period.
    ϕ = x_LPO[2]
     x0_stm = vcat(LPOArrival.x0, reshape(I(6), (36,)))[:]
    _prob = remake(
        LPOArrival.prob_cr3bp_stm;
        tspan = (0.0, ϕ * LPOArrival.period),
        u0 = x0_stm,
        p=[param3b.mu2]
    )
    sol = SailorMoon.integrate_rk4(
        _prob, LPOArrival.dt
    )
    x_tf = sol.u[end][1:6]
    stm = transpose(reshape(sol.u[end][7:end], (6, 6)))

    # translate stable eigenvector and perturb final state
    ys = stm * LPOArrival.ys0
    if ys[1] > 0  # always perturb outward
        state_f = x_tf + LPOArrival.ϵr * ys/norm(ys)
    else
        state_f = x_tf - LPOArrival.ϵr * ys/norm(ys)
    end

    xf = SailorMoon.set_terminal_state2(ϕ, θs[3], param3b, LPOArrival)
    
    println("xf: $xf")
    println("state_f: $state_f")
    
    # transform to Sun-B1 frame
    svf_sunb1 = vcat(
        SailorMoon.transform_EMrot_to_SunB1(state_f, θs[3], param3b.oms, param3b.as),
        x_LPO[3]   # mf
    )
    
    # correct answer
#     svf_sunb1 = [389.90043579649, 0.30945509037997126, 5.600379804037024e-23, -0.3318159893263637, 1.1571761377235485, 3.7612170962908617e-31, x_LPO[3] ]
    
    # ballistic propagation with small time-steps
    params = [
        param3b.mu2, param3b.mus, param3b.as, pi - θs[3], ωM, ωb, 0.0, 0.0, 0.0, mdot, tmax,
        SailorMoon.dv_no_thrust
    ]
    println("params: ", params)
    _prob = remake(
        _prob_base; tspan=[0,-ballistic_time_back], 
        u0 = svf_sunb1, p = params
    )
    sol_ballistic_bck = SailorMoon.integrate_rk4(_prob, 0.001);
    θ_iter = θs[3] - param3b.oms*ballistic_time_back
    
    if verbose
        println("svf_sunb1: \n", svf_sunb1)
    end
    return sol_ballistic_bck.u[end], θ_iter, sol_ballistic_bck
end

In [ ]:
function propagate_arc!(sv0, θ0, tspan, x_control, get_sols::Bool, sol_param_list, name::String)
    sv_iter = [el for el in sv0]
    θ_iter = 1*θ0
    for i = 1:n_arc
        τ, γ, β = x_control[1+3*(i-1) : 3*i]
        
        params = [
            param3b.mu2, param3b.mus, param3b.as, pi - θ_iter, ωM, ωb, τ, γ, β, mdot, tmax,
            SailorMoon.dv_sun_dir_angles
        ]
        _prob = remake(_prob_base; tspan=tspan, u0 = sv_iter, p = params)
        sol = SailorMoon.integrate_rk4(_prob, dt);
        #sol = DifferentialEquations.solve(_prob, RK4(), reltol=1e-10, abstol=1e-10)
        if get_sols
            push!(sol_param_list, [sol, params, name])
        end
        # update θ0 and sv0
        θ_iter += param3b.oms*sol.t[end]
        sv_iter = sol.u[end]
    end
    return sv_iter
end

In [ ]:
multishoot_trajectory = function (x::AbstractVector{T}, get_sols::Bool=false, verbose::Bool=false) where T
    # unpack decision vector
    x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(x)
    
    # initialize storage
    sol_param_list = []
    
    # propagate from LEO forward
    sv0_LEO, θ0_leo, sol_ballistic_fwd = get_LEO_state(x_LEO, θs, verbose)
    svf_LEO = propagate_arc!(
        sv0_LEO, θ0_leo, [0, tofs[1]/n_arc], x_LEO[5 : end],
        get_sols, sol_param_list, "leo_arc"
    )
    
    # propagate midpoint backward
    svm0 = x_mid[1:7]  # state-vector at midpoint, [r,v,mass]
    svf_mid_bck = propagate_arc!(
        svm0, θs[2], [0, -tofs[2]/n_arc], x_mid[10 : end],
        get_sols, sol_param_list, "mid_bck_arc"
    )
    
    # propaagte midpoint forward
    svf_mid_fwd = propagate_arc!(
        svm0, θs[2], [0, tofs[3]/n_arc], x_mid[10+3n_arc : end],
        get_sols, sol_param_list, "mid_fwd_arc"
    )
    
    # propagate from LPO backward
    sv0_LPO, θ0_lpo, sol_ballistic_bck = get_LPO_state(x_LPO, θs, verbose)
    svf_lpo = propagate_arc!(
        sol_ballistic_bck.u[end], θ0_lpo, [0, -tofs[4]/n_arc], x_LPO[5 : end],
        get_sols, sol_param_list, "lpo_arc"
    )
    
    # residuals
    res = vcat(svf_mid_bck - svf_LEO, svf_lpo - svf_mid_fwd)[:]

    # output
    if get_sols == false
        return res
    else
        return res, sol_param_list, [sol_ballistic_fwd,sol_ballistic_bck], tofs
    end
end

In [ ]:
#sol.t[2500], sol.t[end]

In [ ]:
sv_mid = [385.84538133574125, 2.658418835276045, -1.3857413926830354e-25, 0.13274610953763288, 0.0844661017008755, -2.2314313676646742e-26, 1.0]  # mid point state-vector

tof_leo2mid = 13.99899999999768
tof_mid2lpo = tof - tof_leo2mid

# create test decision vector
τ_ig = 0.0
# x_LEO = [ra, Ω, m0, tof, controls...]
ig_x_LEO = vcat(
    [3.8, 0.0, 1.0, tof_leo2mid/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x_mid = vcat(
    sv_mid, tof_leo2mid/2, tof_mid2lpo/2, 
    vcat([[τ_ig,0,0] for i = 1:2n_arc]...)
);
# x_LPO = [θf, ϕ, mf, tof, controls...]
ig_x_LPO = vcat(
    [θsf, ϕ0, 1.0, tof_mid2lpo/2],
    vcat([[τ_ig,0,0] for i = 1:n_arc]...)
);
ig_x = vcat(ig_x_LEO, ig_x_mid, ig_x_LPO);
x_LEO, x_mid, x_LPO, tofs, θs = unpack_x(ig_x, true);

In [ ]:
θsf

In [ ]:
@time hmp, sol_param_list, sols_ballistic, _ = multishoot_trajectory(ig_x, true, true);
hmp
# Any[0.01215058560962404, 328900.5598102475, 388.8212386592645, 0.279252680589793, 0.9251999994040079, 0.07480000059599223, 0.0, 0.0, 0.0, 0.0, 0.0, Main.SailorMoon.dv_sun_dir_angles]

In [ ]:
arcs_color = Dict(
    "leo_arc" => :navy, 
    "mid_bck_arc" => :red1, 
    "mid_fwd_arc" => :darkorange, 
    "lpo_arc" => :green
)

pcart = plot(
    size=(700,500), frame_style=:box, aspect_ratio=:equal, grid=0.4, legend=false
)
plot!(pcart, earth[1,:], earth[2,:], c=:green, lw=1.0, label="earth")
plot!(pcart, earth_ub[1,:], earth_ub[2,:], c=:black, lw=1.0, label="earth")
plot!(pcart, earth_lb[1,:], earth_lb[2,:], c=:black, lw=1.0, label="earth")

plot!(pcart, moon[1,:], moon[2,:], c=:orange, lw=1.0, label="moon")
plot!(pcart, moon_soi_outer[1,:], moon_soi_outer[2,:], c=:black, lw=1.0, label="moon_soi_outer")

plot!(hcat(sol_.u...)[1,:], hcat(sol_.u...)[2,:], color=:black, label="backward(from LLO):0.001", set_aspect=:equal, size=(700,700))


# ballistic legs
for sol_ballistic in sols_ballistic
    plot!(pcart, hcat(sol_ballistic.u...)[1,:], hcat(sol_ballistic.u...)[2,:], c=:dodgerblue, label=false)
end

# trajectory
for i = 1:length(sol_param_list)
    sol, _, name = sol_param_list[i]
    #if name == "mid_bck_arc" || name == "mid_fwd_arc" || name == "lpo_arc"
    plot!(pcart, SailorMoon.Array(sol)[1,:], SailorMoon.Array(sol)[2,:], 
        linewidth=1.5, label="$name", c=arcs_color[name])
    #end
end
# control node
scatter!(pcart, [x_mid[1]], [x_mid[2]], marker=:circle, color=:black, label="MP")
scatter!(pcart, [param3b.as], [0.0], marker=:circle, color=:black)



pcart